In [48]:
class CarInfo:
    def __init__(self, id, model_name: str, body_type: str, fuel_type: str, price: int, power: int, fuel_efficiency: str, model_year: int, size: int, engine_type: str, image_link: str, brand: str):
        self.__id = id
        self.__model_name = model_name
        self.__body_type = body_type
        self.__fuel_type = fuel_type
        self.__price = price
        self.__power = power
        self.__fuel_efficiency
        self.__model_year = model_year
        self.__size = size
        self.__engine_type = engine_type
        self.__image_link = image_link
        self.__brand = brand

    @property # 읽기 허용
    def id(self):
        return self.__id

    @property # 읽기 허용
    def model_name(self):
        return self.__model_name

    @property # 읽기 허용
    def body_type(self):
        return self.__body_type

    @property # 읽기 허용
    def fuel_type(self):
        return self.__fuel_type

    @property # 읽기 허용
    def price(self):
        return self.__price

    @property # 읽기 허용
    def power(self):
        return self.__power

    @property # 읽기 허용
    def fuel_efficiency(self):
        return self.__fuel_efficiency

    @property # 읽기 허용
    def model_year(self):
        return self.__model_year

    @property # 읽기 허용
    def size(self):
        return self.__size

    @property # 읽기 허용
    def engine_type(self):
        return self.__engine_type

    @property # 읽기 허용
    def image_link(self):
        return self.__image_link

    @property # 읽기 허용
    def brand(self):
        return self.__brand

    def __repr__(self):
        return (f'Car_info({self.__id}, {self.__model_name}, {self.__body_type}, {self.__fuel_type}, {self.__price}, {self.__power}, {self.__fuel_efficiency}, {self.__model_year}, {self.__size}, {self.__engine_type}, {self.__image_link}, {self.__brand})')

In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs

# 드라이버 실행
driver = webdriver.Chrome()

# 웹페이지 접속
driver.get("https://naver.com")
time.sleep(3)

# 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("네이버 자동차")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# URL 수집
url_list = []

# 페이지별 컨테이너 > 모델별 상세 페이지 URL 수집
for page in range(1, 54):
    try:
        container = driver.find_element(By.XPATH, f'//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[{page}]')
        car_list = container.find_elements(By.CSS_SELECTOR, ".info_box")

        for car_tag in car_list:
            link = car_tag.find_element(By.CSS_SELECTOR, "a:first-of-type")
            href = link.get_attribute('href')
            if href:
                url_list.append(href + '%20%EC%A0%95%EB%B3%B4')
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

# 모델별 상세 페이지 데이터 수집
car_info_list = []
idx = 1

for url in url_list:
    try:
        driver.get(url)
        time.sleep(3)

        soup = bs(driver.page_source, 'html.parser')

        # 모델명
        model_name_tag = soup.select_one('span.area_text_title')
        model_name = model_name_tag.text.strip() if model_name_tag else None

        # 바디 타입
        body_type_tag = soup.select_one('span.txt:first-of-type')
        body_type = body_type_tag.text.strip() if body_type_tag else None

        # 연식
        model_year_tag = soup.select_one('div.sub_title span.txt:last-of-type')
        model_year = model_year_tag.text.strip() if mode_year_tag else None

        # 스펙 영역 (가격, 파워, 엔진타입, 연비, 사이즈)
        spec_tags = soup.select('div.detail_info')
        info_groups = spec_tags.select('div.info_group')

        # 가격
        price_text = info_groups[0].text.strip()
        price_start = price_text.split('~')[0]
        price = int(price_start.replace(',', '').replace('만원', ''))

        # 연료타입
        fuel_type = info_groups[1].text.strip()

        # 연비
        fuel_efficiency_text = info_groups[2].text.strip()
        fuel_efficiency = fuel_efficiency_text.split('~')[0].replace('복합', '').replace('km/l', '').strip()

        # 파워
        power_text = info_groups[3].text.strip()
        power = power_text.split('/')[0].replace('hp', '').strip()
        power = int(power)

        # 엔진타입
        engine_type = info_groups[7].text.strip()

        # 사이즈
        # 축거
        wheelbase_text = info_groups[-1].text.strip()
        wheelbase = int(''.join(filter(str.isdigit, wheelbase_text)))

        # 전장
        length_text = info_groups[9].text.strip()  # 예: '전장 4,800mm'
        length = int(''.join(filter(str.isdigit, length_text)))

        # 사이즈 계산 (축거 / 전장 * 100)
        size = round((wheelbase / length) * 100, 2)

        # 이미지 링크
        img_tag = soup.select_one('div.thumb img')
        image_link = img_tag['src'] if img_tag else None

        # 브랜드
        brand = mode_name.split(' ')[0] if model_name else None

        car = CarInfo(
            id=idx,
            model_name=model_name,
            body_type=body_type,
            fuel_type=fuel_type,
            price=price,
            power=power,
            fuel_efficiency=fuel_efficiency,
            mode_year=mode_year,
            size=size,
            engine_type=engine_type,
            image_link=image_link,
            brand=brand
        )
        car_info_list.append(car)
        idx += 1

    except Exception as e:
        print(f"URL {url}에서 오류 발생: {e}")

driver.quit()

페이지 3에서 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[3]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010068aa54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000100682cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x00000001001ce864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100215410 cxxbridge1$string$len + 382736
4   chromedriver                        0x0000000100256480 cxxbridge1$string$len + 649088
5   chromedriver                        0x00000001002097ec cxxbridge1$string$len + 334572
6   chromedriver                        0x000000010064fccc cxxbridge1$str$ptr + 2562928
7   chromedriver    

In [54]:
# 결과 출력
print(len(car_info_list))

[]


In [55]:
class CarInfo:
    def __init__(self, id, model_name: str, body_type: str, fuel_type: str, price: int, power: int, fuel_efficiency: str, model_year: int, size: int, engine_type: str, image_link: str, brand: str):
        self.__id = id
        self.__model_name = model_name
        self.__body_type = body_type
        self.__fuel_type = fuel_type
        self.__price = price
        self.__power = power
        self.__fuel_efficiency = fuel_efficiency  # 할당 누락 수정
        self.__model_year = model_year
        self.__size = size
        self.__engine_type = engine_type
        self.__image_link = image_link
        self.__brand = brand

    @property
    def id(self):
        return self.__id

    @property
    def model_name(self):
        return self.__model_name

    @property
    def body_type(self):
        return self.__body_type

    @property
    def fuel_type(self):
        return self.__fuel_type

    @property
    def price(self):
        return self.__price

    @property
    def power(self):
        return self.__power

    @property
    def fuel_efficiency(self):
        return self.__fuel_efficiency

    @property
    def model_year(self):
        return self.__model_year

    @property
    def size(self):
        return self.__size

    @property
    def engine_type(self):
        return self.__engine_type

    @property
    def image_link(self):
        return self.__image_link

    @property
    def brand(self):
        return self.__brand

    def __repr__(self):
        return (f'CarInfo({self.__id}, {self.__model_name}, {self.__body_type}, {self.__fuel_type}, {self.__price}, '
                f'{self.__power}, {self.__fuel_efficiency}, {self.__model_year}, {self.__size}, '
                f'{self.__engine_type}, {self.__image_link}, {self.__brand})')

# -------------------------------------------------------

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs

# 드라이버 실행
driver = webdriver.Chrome()

# 웹페이지 접속
driver.get("https://naver.com")
time.sleep(3)

# 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("네이버 자동차")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# URL 수집
url_list = []

for page in range(1, 54):
    try:
        container = driver.find_element(By.XPATH, f'//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[{page}]')
        car_list = container.find_elements(By.CSS_SELECTOR, ".info_box")

        for car_tag in car_list:
            link = car_tag.find_element(By.CSS_SELECTOR, "a:first-of-type")
            href = link.get_attribute('href')
            if href:
                url_list.append(href + '%20%EC%A0%95%EB%B3%B4')
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

# 모델별 상세 페이지 데이터 수집
car_info_list = []
idx = 1

for url in url_list:
    try:
        driver.get(url)
        time.sleep(3)

        soup = bs(driver.page_source, 'html.parser')

        # 모델명
        model_name_tag = soup.select_one('span.area_text_title')
        model_name = model_name_tag.text.strip() if model_name_tag else None

        # 바디 타입
        body_type_tag = soup.select_one('span.txt:first-of-type')
        body_type = body_type_tag.text.strip() if body_type_tag else None

        # 연식
        model_year_tag = soup.select_one('div.sub_title span.txt:last-of-type')
        model_year = model_year_tag.text.strip() if model_year_tag else None

        # 스펙 영역
        spec_tags = soup.select('div.detail_info')
        if not spec_tags:
            continue
        info_groups = spec_tags[0].select('div.info_group')

        if len(info_groups) < 10:
            continue

        # 가격
        price_text = info_groups[0].text.strip()
        price_start = price_text.split('~')[0]
        price = int(price_start.replace(',', '').replace('만원', ''))

        # 연료타입
        fuel_type = info_groups[1].text.strip()

        # 연비
        fuel_efficiency_text = info_groups[2].text.strip()
        fuel_efficiency = fuel_efficiency_text.split('~')[0].replace('복합', '').replace('km/l', '').strip()

        # 파워
        power_text = info_groups[3].text.strip()
        power = power_text.split('/')[0].replace('hp', '').strip()
        power = int(power)

        # 엔진타입
        engine_type = info_groups[7].text.strip()

        # 사이즈 계산
        wheelbase_text = info_groups[-1].text.strip()
        wheelbase = int(''.join(filter(str.isdigit, wheelbase_text)))

        length_text = info_groups[9].text.strip()
        length = int(''.join(filter(str.isdigit, length_text)))

        size = round((wheelbase / length) * 100, 2)

        # 이미지 링크
        img_tag = soup.select_one('div.thumb img')
        image_link = img_tag['src'] if img_tag else None

        # 브랜드
        brand = model_name.split(' ')[0] if model_name else None

        car = CarInfo(
            id=idx,
            model_name=model_name,
            body_type=body_type,
            fuel_type=fuel_type,
            price=price,
            power=power,
            fuel_efficiency=fuel_efficiency,
            model_year=int(model_year.replace('년', '').strip()) if model_year else None,
            size=size,
            engine_type=engine_type,
            image_link=image_link,
            brand=brand
        )
        car_info_list.append(car)
        idx += 1

    except Exception as e:
        print(f"URL {url}에서 오류 발생: {e}")

driver.quit()

print(len(car_info_list))


페이지 3에서 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[3]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001008d6a54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x00000001008cecf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x000000010041a864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100461410 cxxbridge1$string$len + 382736
4   chromedriver                        0x00000001004a2480 cxxbridge1$string$len + 649088
5   chromedriver                        0x00000001004557ec cxxbridge1$string$len + 334572
6   chromedriver                        0x000000010089bccc cxxbridge1$str$ptr + 2562928
7   chromedriver    